# Home Assignment 1

Send your solutions before 07.10.2021 23:59:59 Moscow time to yanovich.yury@ya.ru. The preferable format of the solution is IPython notebook. Please, set the topic "MMDLS HW1 - \<your last name\>". For example, "MMDLS HW1 - Yanovich". You can write the solution either in Russian or English. Tasks with stars (*) are optional and allow to get bonus points.

In [328]:
# imports
import hashlib
import time
import matplotlib
import networkx as nx
from networkx import balanced_tree, draw_networkx, draw
import numpy as np
import json
import sys

%matplotlib qt5

## Problem 1 (1 point). My Hashrate

This is a simple implemetation of Proof-of-Work function.
It takes current block header and block calculation difficulty as inputs and returns nonce for this block and the header of a mined block.

In [194]:
def proof_of_work(header, target):
    max_nonce = 2 ** 32
    for nonce in range(max_nonce):
        hash_result = hashlib.sha256(str(header).encode('utf-8')+str(nonce).encode('utf-8')).hexdigest()

        if int(hash_result, 16) < target:
            return (hash_result, nonce)

The genesis block is calulated here.

In [196]:
nonce = 0
block_hash = ""
new_block = 'This is the genesis block' + block_hash

difficulty_bits = 8
target = 2 ** (256 - difficulty_bits)

start_time = time.time()
(block_hash, nonce) = proof_of_work(new_block, target)
end_time = time.time()
elapsed_time = end_time - start_time

print("Success with nonce ", nonce)
print("Hash is ", block_hash)
print("Elapsed time: %.4f seconds" % elapsed_time)

Success with nonce  186
Hash is  005eaab75f55d7cef91c03f04052043056113f6ee14abfcf4a0562a019aa1cbe
Elapsed time: 0.0004 seconds


### Task

Choose such a target that you will calculated 20 160 000 blocks per 14 days in avarage. Prove it experimetally (estimate time for at least 2016 blocks and show that it is close to the goal).

*Note.* Each next block should be calculated using the hash of the previous one.

In [197]:
seconds_in_a_day = 60 * 60 * 24
blocks_per_day = 20160000 / 14
estimated_seconds_for_one_block = seconds_in_a_day / blocks_per_day 
estimated_seconds_for_2016_blocks = 2016 * estimated_seconds_for_one_block
print(estimated_seconds_for_2016_blocks)

120.96


In [211]:
# hashrate calculation
header = 'This is the genesis block' + block_hash

start_time = time.time()

for i in range(2**24):
    hash_result = hashlib.sha256(str(header).encode('utf-8')+str(i).encode('utf-8')).hexdigest()
    
end_time = time.time()

elapsed_time = end_time - start_time

hashrate = 2**24 / elapsed_time

In [213]:
print(f'{hashrate:.0f} hashes per second')

523667 hashes per second


In [338]:
difficulty_bits_min: int = 13
difficulty_bits_max: int = 20
    
number_of_blocks = 504

for difficulty_bits in range(difficulty_bits_min, difficulty_bits_max + 1, 1):
    target: int = 2 ** (256 - difficulty_bits)
    print(f'target = 2 ** (256 - {difficulty_bits})')
    new_block: str = 'This is the genesis block'

    start_time = time.time()
    
    for index in range(number_of_blocks): 
        (block_hash, nonce) = proof_of_work(new_block, target)
        new_block = block_hash + str(nonce)
        
    end_time = time.time()

    seconds_for_504_blocks = end_time - start_time
    seconds_for_2016_blocks = (seconds_for_504_blocks / 504) * 2016  
    
    print(f'Seconds for 504 blocks: {seconds_for_504_blocks:.2f}')
    print(f'Estimated time for 2016 blocks (seconds): {seconds_for_2016_blocks:.2f}')
    
    if seconds_for_2016_blocks > np.ceil(estimated_seconds_for_2016_blocks):
        break
    else:
        continue 
    break

target = 2 ** (256 - 13)
Seconds for 504 blocks: 8.28
Estimated time for 2016 blocks (seconds): 33.11
target = 2 ** (256 - 14)
Seconds for 504 blocks: 16.73
Estimated time for 2016 blocks (seconds): 66.93
target = 2 ** (256 - 15)
Seconds for 504 blocks: 35.79
Estimated time for 2016 blocks (seconds): 143.14


In [340]:
difficulty_bits: float = 15
number_of_blocks = 2016
target: int = 2 ** (256 - difficulty_bits)
new_block: str = 'This is the genesis block'

start_time = time.time()

for index in range(number_of_blocks):
    (block_hash, nonce) = proof_of_work(new_block, target)
    new_block = block_hash + str(nonce)
    
end_time = time.time()

seconds_for_2016_blocks = end_time - start_time
print(f'Seconds for 2016 blocks: {seconds_for_2016_blocks:.2f}')
print(f'Estimated time for 20160000 blocks (days): {(seconds_for_2016_blocks * 10000) / (60 * 60 * 24):.0f}')

Seconds for 2016 blocks: 133.90
Estimated time for 20160000 blocks (days): 15


#### I founded the closest target by iterating over values of difficulty_bits. 
#### If we exceed the estimated time for generating 2016 blocks we break the cycle as we have found the value.

#### Actually, If difficulty_bits could be floating point number, we could get closer results, but we could multiply ther target by a constant equal to 1.189 (2 ** (256 - 14.75) - 2 ** (256 - 15))  we could get closer result.

In [359]:
difficulty_bits: float = 15
c: float = 2 ** (256 - 14.75) / 2 ** (256 - difficulty_bits)
number_of_blocks = 2016
target: float = 2 ** (256 - difficulty_bits) * c
new_block: str = 'This is the genesis block'

start_time = time.time()

for index in range(number_of_blocks):
    (block_hash, nonce) = proof_of_work(new_block, target)
    new_block = block_hash + str(nonce)
    
end_time = time.time()

seconds_for_2016_blocks = end_time - start_time
print(f'Seconds for 2016 blocks: {seconds_for_2016_blocks:.2f}')
print(f'Estimated time for 20160000 blocks (days): {(seconds_for_2016_blocks * 10000) / (60 * 60 * 24):.0f}')

Seconds for 2016 blocks: 120.93
Estimated time for 20160000 blocks (days): 14


## Problem 2 (3 + 3* points). Merkle tree

Merkle hash trees play an important role in forming transaction blocks in blockchain. In this assignment we ask you to plot your own Merkle hash tree and check its' properties. Below we provide you with some code fragment what you can use in your assignment

Let us plot graph basis for Merkle hash tree

In [252]:
G = nx.Graph()
positions = {}
coordinates = [
    [0, 4],
    [-2, 3],
    [2, 3],
    [-3, 2],
    [-1, 2],
    [1, 2],
    [3, 2],
    [-3, 1],
    [-1, 1],
    [1, 1],
    [3, 1]
]
parents = [0, 0, 0, 1, 1, 2, 2, 3, 4, 5, 6]
for index in range(11):
    G.add_node(index)
    G.add_edge(index, parents[index])
    positions[index] = coordinates[index]
nx.draw(G, positions, node_size = 1000)
labels = {
    0: '0',
    1: '1',
    2: '2',
    3: '3',
    4: '4',
    5: '5',
    6: '6',
    7: 'tx1',
    8: 'tx2',
    9: 'tx3',
    10: 'tx4',
}
nx.draw_networkx_labels(G, positions, labels = labels)

{0: Text(0, 4, '0'),
 1: Text(-2, 3, '1'),
 2: Text(2, 3, '2'),
 3: Text(-3, 2, '3'),
 4: Text(-1, 2, '4'),
 5: Text(1, 2, '5'),
 6: Text(3, 2, '6'),
 7: Text(-3, 1, 'tx1'),
 8: Text(-1, 1, 'tx2'),
 9: Text(1, 1, 'tx3'),
 10: Text(3, 1, 'tx4')}

In Bitcoin double sha256 hash scheme is used. Here is an example.

In [253]:
first_hash = hashlib.sha256(b"hello") # "b" stands for binary representation
second_hash = hashlib.sha256()
print('First hash represented as a hexadecimal number:', first_hash.hexdigest())
second_hash.update(first_hash.digest())
print('Second hash represented as a hexadecimal number:', second_hash.hexdigest())

First hash represented as a hexadecimal number: 2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824
Second hash represented as a hexadecimal number: 9595c9df90075148eb06860365df33584b75bff782a510c6cd4883a419833d50


Now we can easily change vertices' labels to hashes of corresponding messages and plot new graph

In [254]:
labels[3] = hashlib.sha256(hashlib.sha256(b"tx1").digest()).hexdigest()

# and plot the graph again
nx.draw(G, positions, node_size = 1000)
nx.draw_networkx_labels(G, positions, labels = labels, font_size = 8)

{0: Text(0, 4, '0'),
 1: Text(-2, 3, '1'),
 2: Text(2, 3, '2'),
 3: Text(-3, 2, '856a4921cd32690244af7568e7bd1391a94119e17c7f33234f4bf11271b223e5'),
 4: Text(-1, 2, '4'),
 5: Text(1, 2, '5'),
 6: Text(3, 2, '6'),
 7: Text(-3, 1, 'tx1'),
 8: Text(-1, 1, 'tx2'),
 9: Text(1, 1, 'tx3'),
 10: Text(3, 1, 'tx4')}

1) Construct Merkle hash tree using previously constructed graph by finding corresponding SHA256 hashes on vertices and plot obtained Merkle hash tree (1 point)

Note: for convenience, true hashes have been truncated to the first 6 characters. True hashes are saved in full_labels dictionary.

In [323]:
def construct_merkle_hash_tree(labels: dict) -> (nx.draw_networkx_labels, dict):
    full_labels: dict = labels.copy()
        
    # calculation of values of third level tree nodes
    for i in range(3, 6 + 1):
        full_labels[i] = hashlib.sha256(hashlib.sha256(full_labels[i + 4].encode('utf-8')).digest()).hexdigest()
    # calculation of values of second level tree nodes
    for i in range(1, 2 + 1):
        full_labels[i] = hashlib.sha256(hashlib.sha256(str(full_labels[2 * i + 1] + full_labels[2 * i + 2]).encode('utf-8')).digest()).hexdigest()
    # calculation value of the root node
    full_labels[0] = hashlib.sha256(hashlib.sha256(str(full_labels[1] + full_labels[2]).encode('utf-8')).digest()).hexdigest()
    
    # truncating values to the first 6 characters
    for i in range(6 + 1):
        labels[i] = full_labels[i][:6]
        
    nx.draw(G, positions, node_size = 1000)
    return nx.draw_networkx_labels(G, positions, labels = labels), full_labels


In [324]:
labels: dict = {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: 'tx1', 8: 'tx2', 9: 'tx3', 10: 'tx4'}
draw_networkx_labels, full_labels = construct_merkle_hash_tree(labels)
draw_networkx_labels

{0: Text(0, 4, '5eaee1'),
 1: Text(-2, 3, '41925f'),
 2: Text(2, 3, '5c72c2'),
 3: Text(-3, 2, '856a49'),
 4: Text(-1, 2, '79043a'),
 5: Text(1, 2, 'ef729c'),
 6: Text(3, 2, '4746dc'),
 7: Text(-3, 1, 'tx1'),
 8: Text(-1, 1, 'tx2'),
 9: Text(1, 1, 'tx3'),
 10: Text(3, 1, 'tx4')}

![alt text](first_tree.png "Merkle tree")

2) Provide a proof of correctness of leaf tx1 and set of leafs tx1-tx2 (1 point)

#### proof of correctness of the leaf tx1

In [325]:
leaf_tx1_value: str = 'tx1'
hash_tx2: str = full_labels[4]
hash_tx3_tx4: str = full_labels[2] 

hash_tx1: str = hashlib.sha256(hashlib.sha256(leaf_tx1_value.encode('utf-8')).digest()).hexdigest()
hash_tx1_tx2: str = hashlib.sha256(hashlib.sha256((hash_tx1 + hash_tx2).encode('utf-8')).digest()).hexdigest()
hash_root: str = hashlib.sha256(hashlib.sha256((hash_tx1_tx2 + hash_tx3_tx4).encode('utf-8')).digest()).hexdigest()
    
print(hash_tx1_tx2)
print(hash_root)

assert(hash_root == full_labels[0])

41925fc3630f375e23755f04099f1b2c51f372a9792762b28478fbbc5faf1106
5eaee129fb6c4761a542e919f8fd0fa3f3c3d83a33bbeeec5a7e8772ddfb8f88


#### proof of correctness of the set of leaves tx1 and tx2

In [326]:
leaf_tx1_value: str = 'tx1'
leaf_tx2_value: str = 'tx2'

hash_tx3_tx4: str = full_labels[2] 

hash_tx1: str = hashlib.sha256(hashlib.sha256(leaf_tx1_value.encode('utf-8')).digest()).hexdigest()
hash_tx2: str = hashlib.sha256(hashlib.sha256(leaf_tx2_value.encode('utf-8')).digest()).hexdigest()
hash_tx1_tx2: str = hashlib.sha256(hashlib.sha256((hash_tx1 + hash_tx2).encode('utf-8')).digest()).hexdigest()
hash_root: str = hashlib.sha256(hashlib.sha256((hash_tx1_tx2 + hash_tx3_tx4).encode('utf-8')).digest()).hexdigest()

assert(hash_root == full_labels[0])

3) Change the value on leaf tx1 and recompute corresponding hashes. Plot newly obtained Merkle hash tree (1 point)

In [327]:
labels: dict = {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: 'new_tx1', 8: 'tx2', 9: 'tx3', 10: 'tx4'}
draw_networkx_labels, full_labels = construct_merkle_hash_tree(labels)
draw_networkx_labels

{0: Text(0, 4, 'f59a45'),
 1: Text(-2, 3, 'e11874'),
 2: Text(2, 3, '5c72c2'),
 3: Text(-3, 2, '5ccaca'),
 4: Text(-1, 2, '79043a'),
 5: Text(1, 2, 'ef729c'),
 6: Text(3, 2, '4746dc'),
 7: Text(-3, 1, 'new_tx1'),
 8: Text(-1, 1, 'tx2'),
 9: Text(1, 1, 'tx3'),
 10: Text(3, 1, 'tx4')}

![alt text](second_tree.png "Merkle tree")

4*) Implement a function to support the number of leafs $n =5, .., 16$. Let the value in the leaf $i$ be $txi$. Plot the tree. Provide the proof of correctness of leaf tx4 and set of leafs tx4-tx5 (3 bonus points)

In [ ]:
# code

## Problem 3 (6 + 2* points). Fork rate and main chain

Two miners are the only maintainers of the network. Each miner needs a random time to generate a new block which is modeled as exponential random variables with the rates $1/t_1$ and $1/t_2$ correspondingly. The network is not ideal and it takes a constant time $t_0$ to synchronize the block. We assume both miners be honest and continue the longest known chain.

1) Implement a function to generate a sequence of tuples (time, block author, longest chain, longest chain for miner 1, longest chain for miner 2) and run it for $t_1=t_2=1, t_0=0.5$ (1 point)

In [384]:
class Block(object):
    creation_time: float = 0
    author: str = ''
    
    def __init__(self, creation_time: int, author: str):
        self.creation_time = creation_time
        self.author = author

    def __repr__(self):
        return f'creation_time = {self.creation_time}; author = {self.author}'

#### Let's assume that our miners started generating their blocks at the same time. (I created a class for block, hope it is not a problem.)

In [398]:
def get_chains(t1: float, t2: float, t0: float) -> ([], [], []):
    time_first: float = 0
    time_second: float = 0
    first_miner_chain = []
    second_miner_chain = []
    chain = []

    for i in range(10):
        # first miner mines block
        first_miner_time = round(np.random.exponential(scale = 1 / t1), 3)
        # second miner mines block
        second_miner_time = round(np.random.exponential(scale = 1 / t2), 3)

        # time after start mining [first miner]
        time_first += first_miner_time
        time_first = round(time_first, 3)
        # time after start mining [second miner]
        time_second += second_miner_time
        time_second = round(time_second, 3)

        # if first miner was quicker
        if time_first < time_second:
            # create block
            first_miner_block = Block(time_first, 'first_miner')
            # apppend block to first miner chain
            first_miner_chain.append(first_miner_block)        
            # now chain is first_miner_chain
            chain = first_miner_chain
            # if second miner saw a new block before he ended mining his block
            if time_second - time_first > t0:
                # let's say that he stopped mining this block and started mining new
                time_second -= (second_miner_time - first_miner_time - t0)  
                time_second = round(time_second, 3)
                second_miner_chain = chain
            # if second miner did not see new block before he ended mining, append block to second miner chain
            else:
                second_miner_block = Block(time_second, 'second_miner')
                second_miner_chain.append(second_miner_block)
        # if second miner was quicker        
        else:
            # create block
            second_miner_block = Block(time_second, 'second_miner')
            # append block to second miner chain
            second_miner_chain.append(second_miner_block)        
            # now chain is second_miner_chain
            chain = second_miner_chain
            # if first miner saw a new block before he ended minin his block
            if time_first - time_second > t0:
                # let's say that he stopped mining this block and started mining new
                time_first -= (first_miner_time - second_miner_time - t0)
                time_first = round(time_first, 3)
                first_miner_chain = chain
            # if first miner did not see new block before he ended mining, append block to first miner chain:
            else:   
                first_miner_block = Block(time_first, 'first_miner')
                first_miner_chain.append(first_miner_block)
    
    return chain, first_miner_chain, second_miner_chain

In [399]:
chain, first_miner_chain, second_miner_chain = get_chains(1, 1, 0.5)

In [400]:
chain

[creation_time = 0.827; author = first_miner,
 creation_time = 1.732; author = first_miner,
 creation_time = 2.053; author = second_miner,
 creation_time = 2.358; author = second_miner,
 creation_time = 2.591; author = first_miner,
 creation_time = 3.644; author = second_miner,
 creation_time = 4.952; author = first_miner,
 creation_time = 5.915; author = first_miner,
 creation_time = 6.129; author = second_miner,
 creation_time = 6.244; author = first_miner,
 creation_time = 6.738; author = second_miner,
 creation_time = 8.256; author = first_miner,
 creation_time = 8.268; author = first_miner,
 creation_time = 10.13; author = second_miner]

2) Plot the ratio of the blocks in the main chain to the number of generated blocks as a function of $t_0 \in [0, 2]$ for fixed $t_1=t_2=1$ (2 points)

In [6]:
# code

3) Let there be an adversary miner 3 with a rate $t_3 = 0.9$ and instant communication with miners 1 and 2. Let him mine his own branch. Plot the empirical and theoretical probability to go $6$ blocks ahead of the miners' 1 and 2 main chain as a function of $t_0$ for fixed $t_1=t_2=1$ (3 points)

In [7]:
# code

4*) Demonstrate double spend attack by miner 3 (2 bonus points)

In [8]:
# code